In [ ]:
# conda activate anndata

import os
import sys

sys.path.append("/mnt/lareaulab/reliscu/code")

from junction2psi import *

In [2]:
pseudobulk_data = "tasic_2018_ALM_STAR_SyntheticDataset1_20pcntCells_25SD_200samples_SJ_pseudobulk"
sj_file = f"data/{pseudobulk_data}.csv"

In [3]:
min_observed = 0.25
minPsi = 0.1

In [4]:
SJ_counts_table = pd.read_csv(sj_file, index_col=0)

events_i1 = pd.Index([x[:-3] for x in SJ_counts_table.index if '_I1' in x])
events_i2 = pd.Index([x[:-3] for x in SJ_counts_table.index if '_I2' in x])
events_se = pd.Index([x[:-3] for x in SJ_counts_table.index if '_SE' in x])

events = events_i1.intersection(events_i2).intersection(events_se)
i1_events = [x + '_I1' for x in events]
I1_table = SJ_counts_table.loc[i1_events]
I1_table.index = events

i2_events = [x + '_I2' for x in events]
I2_table = SJ_counts_table.loc[i2_events]
I2_table.index = events

se_events = [x + '_SE' for x in events]
SE_table = SJ_counts_table.loc[se_events]
SE_table.index = events

I1_filt = I1_table.index[I1_table.sum(axis=1) > 0]
I2_filt = I2_table.index[I2_table.sum(axis=1) > 0]
SE_filt = SE_table.index[SE_table.sum(axis=1) > 0]
filtered_events = I1_filt.intersection(I2_filt).intersection(SE_filt)

I1_table = I1_table.loc[filtered_events]
I2_table = I2_table.loc[filtered_events]
SE_table = SE_table.loc[filtered_events]

psi = ((I1_table + I2_table) /(2*SE_table + I1_table + I2_table)).fillna(0)
reads = SE_table + I1_table + I2_table

In [5]:
alt_exons = psi.index[np.abs(0.5-psi.mean(axis=1)) <= (0.5-minPsi)]
obs_exons = psi.index[psi.isna().mean(axis=1) <= 1-min_observed]
selected_exons = alt_exons.intersection(obs_exons)

In [6]:
len(selected_exons)

5467

In [7]:
psi.shape[0]

14924

In [8]:
psi = psi.loc[selected_exons]
reads = reads.loc[selected_exons]

In [9]:
psi.to_csv(f"data/{pseudobulk_data}_min_observed{min_observed}_minPsi{minPsi}_PSI.csv")
reads.to_csv(f"data/{pseudobulk_data}_min_observed{min_observed}_minPsi{minPsi}_counts.csv")